In [1]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import paramiko

/Users/Meredith/Applications/Ureka/variants/common/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
/Users/Meredith/Applications/Ureka/variants/common/lib/python2.7/site-packages/Crypto/Util/number.py:57: PowmInsecureWarning: Not using mpz_powm_sec.  You should rebuild using libgmp >= 5 to avoid timing attack vulnerability.
  _warn("Not using mpz_powm_sec.  You should rebuild using libgmp >= 5 to avoid timing attack vulnerability.", PowmInsecureWarning)


This is specifically for analyzing 1-meter photometry data, which I have stored in a specific place.

In [2]:
fullFilePath = '/mnt/mrawls/1m_obs/'
hostname = 'seismo'
username = 'mrawls'		# password is magically saved in SSH file-land
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.load_system_host_keys()
try:		# maybe we're on campus
	client.connect(hostname, 22, username)
	print('success! connected')
except:		# maybe we're not on campus
	print('connecting to astronomy first')
	client.connect('astronomy.nmsu.edu', 22, username)
	proxy_command = 'ssh %s@astronomy.nmsu.edu nc %s %s' % (username, hostname+'.nmsu.edu', 22)
	proxy = paramiko.ProxyCommand(proxy_command)
	client = paramiko.SSHClient()
	client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
	client.connect(hostname, 22, username, sock=proxy)
	print('success! connected')

connecting to astronomy first
success! connected


In [3]:
sftp = client.open_sftp()
listOfAllFiles = sftp.listdir(fullFilePath)	# get all files and directories
listOfDirs = []
for item in listOfAllFiles:					# get directories that are dates (eg '140101')
	if item[0:2] == ('14' or '15'):
		listOfDirs.append(fullFilePath + str(item) + '/')

In [15]:
listOfFitsFiles = []
#test = sftp.file('/mnt/mrawls/1m_obs/140522/140522.108.fits')
#print(test)
#print(listOfDirs)
for dir in listOfDirs[1:]: # THIS IS BREAKING... it can only access the first dir in this list for some reason.
    try:
        listOfFilesInDir = sftp.listdir(dir)
        print('successfully accessed {0} directory'.format(dir))
        for file in listOfFilesInDir:
            if file[-4:] == ('fits' or 'FITS'):
                fullfilepath = dir+file
                try:
                    sftp.file(fullfilepath)
                    listOfFitsFiles.append(sftp.file(fullfilepath))
                except:
                    #print('Can\'t access file {0} with SFTP'.format(fullfilepath))
                    continue
    except:
        print('skipping {0} directory'.format(dir))
        #continue

#print(fits.open(listOfFitsFiles[0], ignore_missing_end=True)[0].header) #example
for object in listOfFitsFiles:
    hdu = fits.open(object, ignore_missing_end = True)
    print(hdu[0].header['date-obs'], hdu[0].header['RA'], hdu[0].header['filtname'][-9])

successfully accessed /mnt/mrawls/1m_obs/140628/ directory
skipping /mnt/mrawls/1m_obs/140629/ directory
skipping /mnt/mrawls/1m_obs/140630/ directory
skipping /mnt/mrawls/1m_obs/140829/ directory
skipping /mnt/mrawls/1m_obs/140831/ directory
skipping /mnt/mrawls/1m_obs/140901/ directory
skipping /mnt/mrawls/1m_obs/140902/ directory
skipping /mnt/mrawls/1m_obs/140903/ directory
skipping /mnt/mrawls/1m_obs/140910/ directory
skipping /mnt/mrawls/1m_obs/140414/ directory
skipping /mnt/mrawls/1m_obs/140418/ directory
skipping /mnt/mrawls/1m_obs/140421/ directory
skipping /mnt/mrawls/1m_obs/140422/ directory
skipping /mnt/mrawls/1m_obs/140424/ directory
skipping /mnt/mrawls/1m_obs/140425/ directory
skipping /mnt/mrawls/1m_obs/140429/ directory
skipping /mnt/mrawls/1m_obs/140430/ directory
skipping /mnt/mrawls/1m_obs/140504/ directory
skipping /mnt/mrawls/1m_obs/140507/ directory
skipping /mnt/mrawls/1m_obs/140508/ directory
skipping /mnt/mrawls/1m_obs/140509/ directory
skipping /mnt/mrawls/